### Importando as bibliotecas

In [3]:
import pandas as pd
from pathlib import Path
from gtin_validation import main
from src.normalize_function import normalized_gtin

### Carregando o arquivo

In [4]:
file_path = Path.cwd() / "data" / "base_total_v2.xlsx"
df = pd.read_excel(file_path)
df.describe(include=object)

,NOME,BARCODE,BARCODE_NF
count,82546,82547,82548
unique,81638,81383,77232
top,Carne Short Rib CARAPRETA no kg,\N,SEM GTIN
freq,18,849,4687


In [5]:
df['BARCODE'] = df['BARCODE'].astype(str)
df['BARCODE_NF'] = df['BARCODE_NF'].astype(str)
#df.rename(columns={'BARCODE': 'COD. DE BARRAS'}, inplace=True)

### Funções

In [6]:
print(f'total produtos: {df['ID'].count()}')

total produtos: 82548


In [7]:
df['valido'] = df['BARCODE'].apply(main)
df['normalizado'] = df['BARCODE'].apply(normalized_gtin)
df['valido_normalizado'] = df['normalizado'].apply(main)
df['duplicado'] = df['BARCODE'].duplicated()
df['duplicado_normalizado'] = df['normalizado'].duplicated()
df['BARCODE_NF2'] = df.apply(lambda row: row['normalizado'] if main(row['normalizado']) else "SEM GTIN", axis=1)
#df.to_excel('teste.xlsx', index=False)

In [8]:
print(f"total validos: {df['valido'].sum()}")
print(f"total invalidos: {(df['valido'] == False).sum()}")
print(f"total validos normalizado: {df['valido_normalizado'].sum()}")
print(f"total invalidos normalizado: {(df['valido_normalizado'] == False).sum()}")
print(f"total duplicados: {df['duplicado'].sum()}")
print(f"total duplicados normalizado: {df['duplicado_normalizado'].sum()}")

total validos: 77271
total invalidos: 5277
total validos normalizado: 77868
total invalidos normalizado: 4680
total duplicados: 1164
total duplicados normalizado: 1915


In [8]:
df.to_excel('yyyyyyyyyyyy.xlsx')

### Montando um DataFrame para fazer as análises

In [5]:
products_df = df[df['FAT_2024'] != 0].copy()
products_df = products_df.sort_values(by=['FAT_2024'], ascending=False)
products_df['COD. DE BARRAS SO DIGITO'] = products_df['COD. DE BARRAS'].apply(raw_gtin)
products_df['COD. DE BARRAS NORMALIZADO'] = products_df['COD. DE BARRAS SO DIGITO'].apply(normalized_gtin)
products_df['DUPLICADO'] = products_df['COD. DE BARRAS'].duplicated()
products_df['DUPLICADO SO DIGITO'] = products_df['COD. DE BARRAS SO DIGITO'].duplicated()
products_df['VAL. COD. DE BARRAS'] = products_df['COD. DE BARRAS'].apply(main)
products_df['VAL. COD. DE BARRAS NORMALIZADO'] = products_df['COD. DE BARRAS NORMALIZADO'].apply(main)
products_df['id'] = pd.factorize(products_df['COD. DE BARRAS NORMALIZADO'])[0]

#AMBOS VALIDANDO - 25802
#COM O CADASTRADO NO SISTEMA COMO VALIDADADO E COM O NORMALIZADO COMO INVALIDO - 0
#COM O CADASTRADO NO SISTEMA COMO INVALIDO E COM O NORMALIZADO COMO VALIDADADO - 308
#AMBOS INVALIDOS - 446

filtered_df = products_df[(products_df['COD. DE BARRAS SO DIGITO'] == '70847033301')]
filtered_df.head()

,ID_PDX,PRODUTO,NOME,COD. DE BARRAS,FAT_2024,COD. DE BARRAS SO DIGITO,COD. DE BARRAS NORMALIZADO,DUPLICADO,DUPLICADO SO DIGITO,VAL. COD. DE BARRAS,VAL. COD. DE BARRAS NORMALIZADO,id
3054,13195,45473,Energético MONSTER Mango Loco 473ml,70847033301,521383.81,70847033301,0070847033301,False,False,False,True,24
4298,13653,47169,Energético MONSTER Mango loco Lata 473ml,070847033301e,4508.14,70847033301,0070847033301,False,True,False,True,24
33603,15147,150631,Energético juice monster mango loco lata 473ml,70847033301,13.07,70847033301,0070847033301,True,True,False,True,24


## 2 - Análise dos produtos e duplicados

### Separando o df em únicos e duplicados

In [6]:
unique_prod_df = products_df[(products_df['DUPLICADO SO DIGITO'] == False)]
duplicates_prod_df = products_df[(products_df['DUPLICADO SO DIGITO'])]

### Criando um df com os códigos de barras duplicados e seus respectivos faturamentos

In [7]:
fat_dup_df = duplicates_prod_df.groupby('COD. DE BARRAS SO DIGITO', as_index=False)['FAT_2024'].sum().sort_values('FAT_2024', ascending=False)
fat_dup_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 44 to 52
Data columns (total 2 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   COD. DE BARRAS SO DIGITO  99 non-null     object 
 1   FAT_2024                  99 non-null     float64
dtypes: float64(1), object(1)
memory usage: 2.3+ KB


### Representatividade de duplicados no faturamento total

In [8]:
fat_total = products_df['FAT_2024'].sum()
fat_total_duplicados = fat_dup_df['FAT_2024'].sum()
duplicados_perc = ((fat_total_duplicados/fat_total)*100).round(2)
total_duplicados = len(duplicates_prod_df)
print(f'O total de produtos duplicados são {total_duplicados}')
print(f'O Faturamento deles representam {duplicados_perc} % do faturamento total')


O total de produtos duplicados são 109
O Faturamento deles representam 0.27 % do faturamento total


### Montando um df com os produtos "Pai" e informações de faturamentos dos produtos "Filhos"
Afim de fazer uma análise representatividade dos duplicados em relação ao produto pai 

In [9]:
unique_prod_df = unique_prod_df.merge(fat_dup_df, how='left', left_on=['COD. DE BARRAS SO DIGITO'], right_on=['COD. DE BARRAS SO DIGITO'])
unique_prod_df.fillna(0)
unique_prod_df['PERC_FAT_DUP'] = (unique_prod_df['FAT_2024_y']/(unique_prod_df['FAT_2024_x'] + unique_prod_df['FAT_2024_y']))*100
unique_prod_df = unique_prod_df.rename(columns={'FAT_2024_x': 'FAT_2024', 'FAT_2024_y': 'FAT_2024_DUP'})
unique_prod_df.head()

,ID_PDX,PRODUTO,NOME,COD. DE BARRAS,FAT_2024,COD. DE BARRAS SO DIGITO,COD. DE BARRAS NORMALIZADO,DUPLICADO,DUPLICADO SO DIGITO,VAL. COD. DE BARRAS,VAL. COD. DE BARRAS NORMALIZADO,id,FAT_2024_DUP,PERC_FAT_DUP
0,13195,41466,"Refrigerante COCA-COLA Original 1,5L",7894900011753,2025993.13,7894900011753,7894900011753,False,False,True,True,0,NaN,NaN
1,13653,43794,"Refrigerante COCA-COLA Sem Açúcar 1,5L",7894900701753,1956854.94,7894900701753,7894900701753,False,False,True,True,1,NaN,NaN
2,13195,37137,Cerveja HEINEKEN Long Neck 330ml,78936683,1617171.09,78936683,78936683,False,False,True,True,2,512.24,0.031665
3,13195,36714,Refrigerante COCA-COLA Original 2L,7894900027013,1476378.79,7894900027013,7894900027013,False,False,True,True,3,NaN,NaN
4,13195,153149,Cerveja HEINEKEN Lata Sleek 350ml,7896045506873,1086855.90,7896045506873,7896045506873,False,False,True,True,4,NaN,NaN


### Análise de representatividade dos itens não duplicados que estão com o GTIN inválido

In [10]:
invalid_unique_prod_df = unique_prod_df[(unique_prod_df['VAL. COD. DE BARRAS NORMALIZADO'] == False)]
fat_total_invalidos = invalid_unique_prod_df['FAT_2024'].sum()
fat_total_unicos = unique_prod_df['FAT_2024'].sum()
invalid_perc = ((fat_total_invalidos/fat_total_unicos)*100).round(2)
total_invalidos = invalid_unique_prod_df['PRODUTO'].count()
print(f'O total de produtos únicos inválidos são {total_invalidos}')
print(f'O faturamento deles representam {invalid_perc} % do faturamento total')


O total de produtos únicos inválidos são 446
O faturamento deles representam 0.12 % do faturamento total


In [11]:
qty_unique_id = products_df['id'].nunique()
qty_total = products_df['id'].count()
print(f'Quantidade total: {qty_total}')
print(f'Quantidade única: {qty_unique_id}')
print(f'Quantidade duplicados: {qty_total-qty_unique_id}')

Quantidade total: 26556
Quantidade única: 26447
Quantidade duplicados: 109


In [12]:
duplicates_id = products_df.groupby('id').filter(lambda x: len(x) > 1)['id']
duplicates_id = duplicates_id.unique()
only_duplicates_df = products_df[products_df['id'].isin(duplicates_id)]
only_duplicates_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 208 entries, 217 to 5951
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   ID_PDX                           208 non-null    int64  
 1   PRODUTO                          208 non-null    int64  
 2   NOME                             208 non-null    object 
 3   COD. DE BARRAS                   208 non-null    object 
 4   FAT_2024                         208 non-null    float64
 5   COD. DE BARRAS SO DIGITO         208 non-null    object 
 6   COD. DE BARRAS NORMALIZADO       208 non-null    object 
 7   DUPLICADO                        208 non-null    bool   
 8   DUPLICADO SO DIGITO              208 non-null    bool   
 9   VAL. COD. DE BARRAS              208 non-null    bool   
 10  VAL. COD. DE BARRAS NORMALIZADO  208 non-null    bool   
 11  id                               208 non-null    int64  
dtypes: bool(4), float64(1), 

In [13]:
output_path = Path.cwd() / "data" / "unique_prod.xlsx"
unique_prod_df.to_excel(output_path, index=False)